In [1]:
from solve_lindblad import *
import qutip as qu

from scipy.linalg import expm
import scipy

In [2]:
from derivatives import compute_probability

In [3]:
final_t = 1.2

omega = 10
J = 0.8
H0 = omega * qu.sigmax()
psi0 = qu.basis(2, 0)
rho0 = qu.ket2dm(psi0)

In [19]:
%%timeit
evolve_dm(rho0, H0, [J * qu.sigmax()], final_t, 1)


1.24 ms ± 72.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [22]:

def my_evolve_dm(rho0, H0, operators, t, no_qubits=1):
    liouvillian = qu.liouvillian(H0, c_ops=operators)
    rho_sup = qu.to_super(rho0)
    rho_evol = scipy.linalg.expm(liouvillian.full()*t)@rho_sup.full()
    dm_evol = rho_evol[:, 0].reshape(-1, 2**no_qubits)
    dm_evol = qu.Qobj(dm_evol)
    return dm_evol

In [23]:
%%timeit
my_evolve_dm(rho0, H0, [J * qu.sigmax()], final_t, 1)


1.09 ms ± 23.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [2]:
a = qu.sigmax()

In [3]:
%%timeit 
qu.sigmax()

17.2 ns ± 0.32 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


In [4]:
%%timeit 
qu.sigmax()

205 µs ± 14.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [11]:
class a:
    def __init__(self):
        self.p = 2
    @property
    def piernas(self):
        return 2

In [12]:
b = a()

In [14]:
%%timeit
b.piernas

71 ns ± 1.75 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [15]:
%%timeit
b.p

39.7 ns ± 0.981 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [4]:
import custom_models


In [5]:
a = custom_models.SimplePrecessionWithNoise()

In [7]:
a.UnitH0

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0. 1.]
 [1. 0.]]

dtype('float64')

In [6]:
import numpy as np
from evolution import evolve_dm
import qutip as qu

In [46]:
rho = evolve_dm(a.SuperRho0, a.UnitH0, 0.3*qu.sigmax(), 0.3)

In [7]:
%%timeit
evolve_dm(a.SuperRho0, a.UnitH0, 0.3*qu.sigmax(), 0.3)

1.1 ms ± 58.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [43]:
%%timeit
for i in range(10):
    evolve_dm(a.SuperRho0, a.UnitH0, 0.3*qu.sigmax(), 0.3)

10.4 ms ± 231 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [50]:
import cython
%load_ext Cython


In [40]:
%%cython

from evolution import evolve_dm
def hola(int reps, *args):
    for i in range(reps):
        evolve_dm(*args)

In [42]:
%%timeit
hola(10, a.SuperRho0, a.UnitH0, 0.3*qu.sigmax(), 0.3)

7.77 ms ± 546 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [53]:
import derivatives

In [10]:
import evolution

In [12]:
projs = [np.array([[1, 0], [0, 0]]), np.array([[0, 0], [0, 1]])]

In [15]:
%%timeit
compute_probability(rho0.full(), projs)

21.5 µs ± 605 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [16]:
%%timeit
qu.measurement.measurement_statistics(rho0, qu.sigmaz())

1.39 ms ± 56.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [17]:
p0 = compute_probability(rho0.full(), projs)

In [18]:
p0

array([1., 0.])

In [19]:
p0[0]

1.0

In [74]:
from qinfer.tests import test_model
from qinfer import UniformDistribution
from custom_models import SimplePrecessionWithNoise
import numpy as np


model = SimplePrecessionWithNoise()
prior = UniformDistribution([[0, 0.5], [0, 0.5]])
prior.sample()


modelparams = np.dstack(np.mgrid[0:1:100j, 0:1:100j]).reshape(-1, 2)


expparams = np.empty((9,), dtype=model.expparams_dtype)
expparams["t"] = np.dstack(np.mgrid[1:10] * np.pi / 2)


# test_model(model, prior, expparams)


In [76]:
import itertools
all_possible_exps = list(itertools.product(expparams["t"], modelparams))
# a.likelihood([0, 1], modelparams, expparams)

In [39]:
%%timeit
aux1 = qu.parallel_map(a.single_qu_likelihood, all_possible_exps)

815 ms ± 44.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
%%timeit
aux2 = qu.parfor(a.single_qu_likelihood, all_possible_exps)

714 ms ± 31.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
aux1 = qu.parallel_map(a.single_qu_likelihood, all_possible_exps)

In [45]:
aux2 = qu.parfor(a.single_qu_likelihood, all_possible_exps)

In [48]:
aux2 == aux1

True

In [64]:
%%cython 

from cython.parallel import prange

def run(a, exps):
    re = []
    n = len(exps)

    for i in prange(n, nogil=True):
    # for i in exps:
        re.append(a.single_qu_likelihood(exps[i]))
    return re



# def func(double[:] x, double alpha):
#     cdef Py_ssize_t i

#     for i in prange(x.shape[0], nogil=True):
#         x[i] = alpha * x[i]    


Error compiling Cython file:
------------------------------------------------------------
...

def run(a, exps):
    re = []
    n = len(exps)

    for i in prange(n, nogil=True):
       ^
------------------------------------------------------------

/home/antonio/.cache/ipython/cython/_cython_magic_956f0184063b60e46a6b8ccaf9316eb0.pyx:8:8: target may not be a Python object as we don't have the GIL

Error compiling Cython file:
------------------------------------------------------------
...
    re = []
    n = len(exps)

    for i in prange(n, nogil=True):
    # for i in exps:
        re.append(a.single_qu_likelihood(exps[i]))
                ^
------------------------------------------------------------

/home/antonio/.cache/ipython/cython/_cython_magic_956f0184063b60e46a6b8ccaf9316eb0.pyx:10:17: Calling gil-requiring function not allowed without gil

Error compiling Cython file:
------------------------------------------------------------
...
    re = []
    n = len(exps)

    for 

In [60]:
%%timeit
run(a, all_possible_exps)

2.11 s ± 33 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [54]:
all_possible_exps[0]

(1.5707963267948966, array([0., 0.]))

In [78]:
from multiprocessing import Pool

re = []
with Pool() as pool:
    # call the same function with different data in parallel
    for result in pool.map(a.single_qu_likelihood, all_possible_exps):
        re.append(result)
        # report the value to show progress

In [71]:
re ==aux1

True

In [77]:
qu.parfor(a.single_qu_likelihood, all_possible_exps)

[array(1.),
 array(0.99983976),
 array(0.99935933),
 array(0.99855966),
 array(0.99744226),
 array(0.99600929),
 array(0.99426348),
 array(0.99220817),
 array(0.98984728),
 array(0.98718529),
 array(0.98422724),
 array(0.98097871),
 array(0.97744579),
 array(0.97363509),
 array(0.96955369),
 array(0.96520913),
 array(0.96060939),
 array(0.95576286),
 array(0.95067831),
 array(0.94536488),
 array(0.93983203),
 array(0.93408955),
 array(0.92814748),
 array(0.92201612),
 array(0.91570598),
 array(0.90922776),
 array(0.90259232),
 array(0.89581064),
 array(0.88889381),
 array(0.88185298),
 array(0.87469933),
 array(0.86744407),
 array(0.86009836),
 array(0.85267334),
 array(0.84518006),
 array(0.83762948),
 array(0.83003242),
 array(0.82239955),
 array(0.81474137),
 array(0.80706819),
 array(0.79939008),
 array(0.79171689),
 array(0.78405819),
 array(0.7764233),
 array(0.76882123),
 array(0.76126069),
 array(0.75375006),
 array(0.74629739),
 array(0.73891041),
 array(0.73159645),
 array(0.

In [83]:
from dask import delayed



ModuleNotFoundError: No module named 'yaml'

In [ ]:
for x in data:
    y = delayed(inc)(x)
    results.append(y)